In [18]:
%load_ext autoreload
%autoreload

import pandas as pd
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
import texcrapy
from ckonlpy.tag import Twitter
import json

#import os
#import numpy as np
#import asyncio
#from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 아이템별 쿼리데이터(dict) 만들기

In [2]:
df = pd.read_excel('keywords and logos.xlsx', sheet_name='20190215')[['shortname','kw_supporter','kw_supported','keywords']]; df
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

qry_base = {row.shortname:_or(row.keywords) for row in df.itertuples()}
supporters = df.shortname[df.kw_supporter==True]
qry_sup = ' OR '.join([qry_base[sup] for sup in supporters]); qry_sup
qry = {row.shortname: '(' + qry_base[row.shortname] + ') AND (' + qry_sup + ')' if row.kw_supported==True else qry_base[row.shortname] for row in df.itertuples()}

In [3]:
qry

{'ootd': '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏',
 'category': '#designer OR #streetwear OR #luxury OR #casual OR #sportswear OR #디자이너 OR #길거리패션 OR #럭셔리 OR #캐주얼 OR #캐쥬얼 OR #스포츠웨어 OR #스포츠패션',
 'fashion': '#fashion OR #brand OR #fashionbrand OR #shoe OR #shoes OR #clothes OR #wear OR #apparel OR #패션 OR #브랜드 OR #패션브랜드 OR #옷 OR #의류 OR #슈즈 OR #신발',
 '8seconds': '#8seconds OR #8second OR #에잇세컨즈 OR #8세컨즈',
 'adidas': '#adidas OR #아디다스',
 'adidasoriginal': '#adidasoriginal OR #아디다스오리지널스 OR #아디다스오리지널 OR #아디다스오리지날스 OR #아디다스오리지날',
 'asics': '#asics OR #아식스',
 'balenciaga': '#balenciaga OR #balensiaga OR #발렌시아가',
 'bally': '#bally OR #발리',
 'beanpole': '#beanpole OR #빈폴',
 'benetton': '#benetton OR #베네통',
 'blackyak': '#blackyak OR #블랙야크',
 'buckaroo': '#buckaroo OR #버커루',
 'burberry': '#burberry OR #버버리',
 'calvinklein': '#calvinklein OR #케빈클라인 OR #켈빈클라인 OR #캘빈클라인',
 'canadagoose': '#canadagoose OR #캐나다구스',
 'chanel':

In [6]:
# 요게 필요한가?
with open('twitter_data/qry.json', 'w') as f:
    json.dump(qry, f)

In [17]:
%%time
what = ['id', 'timestamp', 'text']
texcrapy.scrap(qry, what=what, lang='ko', end='2019-01-31', download_to='scrapped/twitter')

INFO: queries: ['#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-01-01 until:2010-06-15', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-06-15 until:2010-11-28', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-11-28 until:2011-05-13', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2011-05-13 until:2011-10-26', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2011-10-26 until:2012-04-09', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2012-04-09 until:2012-09-22', '#dailylook OR #outfit OR #bestofstreetwear OR #bestst


Wall time: 5h 1min 17s


# 사전 만들기

In [77]:
newwords_kw = [w.strip() for w in ', '.join(df.keywords).split(',')]
newwords_custom = [
    '매거진','1박2일','아우터','항공점퍼','청치마','맨투맨','응답하라','추억팔이','레드벨벳','배꼽티','켄달제너','패션템','스타일리쉬',
    '스타일리시','나들이룩','섹시','섹시미','스타일링','입소문','비현실','세일','레스모아','요거','무개념','보좌진','언제나','간신히','가성비',
    'g1','10k','워킹화','쿠션감','후원금','기업','오니츠카','오니츠카타이거','농구화','황금비율','대다나다','일본기업',
    '슬립온','맥시코','데일리','착화감','마라톤화','젤라이트','너무','예뻐','젤카야노','카야노','신박','이쁜','런닝화','러닝화','체리핑크','퓨젝스',
    '골프화','자세히','유투버','신상','신상품','던롭','국대','단체복','스티브잡스','빅토리아시크릿','잡스타그램','a6','테스트모니얼',
    '스트릿패션','스트릿브랜드','굿즈','베리베리','반팔티','지급품','직영점','패셔니스타','패션피플','런스타그램','리미티드에디션','한동안',
    '어글리','젤버즈','아이스버킷챌린지','아이스버킷','챌린지','메탈릭','활력'
]
newwords = list(set(newwords_kw + newwords_custom))

In [78]:
twitter = Twitter()
twitter.add_dictionary(newwords, 'Noun')

### text = 트위터 글 하나하나
### doc = 아이템별 text 들의 모임

In [81]:
with open('scrapped/twitter/asics (2010.01.01-2019.01.31).json', encoding='UTF-8-sig') as f:
    jsn = json.load(f)
    
%time toks = texcrapy.tokenize_docs(*jsn['asics'], with_pos=False, tagger=twitter)
toks


Wall time: 10.5 s


[['족적', 'jpg', 'asics'],
 ['12월',
  '둘째',
  '주',
  '이벤트',
  '스포츠',
  '브랜드',
  '방한',
  '대전',
  '겨울',
  '요거',
  '하나',
  '딱',
  '있다',
  '엄청',
  '따다',
  '수운',
  '디',
  '안',
  '볼텨',
  '아식스',
  '뉴발란스',
  '르꼬끄',
  '나이키',
  '데상트',
  '리복',
  '패딩',
  '다운',
  '지금',
  '만나다',
  '보',
  '세용',
  '퐁당'],
 ['세일',
  '퐁당',
  '오',
  '포도',
  '몰다',
  '이렇다',
  '세일',
  '혼자',
  '알',
  '있다',
  '까운',
  '유명',
  '브랜드',
  'sale',
  '소식',
  '알다',
  '드립',
  '다',
  '아식스',
  '레스모아',
  '나이키',
  '숲',
  '미샤',
  '등'],
 ['박근혜',
  '깜짝',
  '시구',
  '글러브',
  '태극마크',
  '운동화',
  '일제',
  '아식스',
  '이투데이',
  '아베',
  '오빠',
  '날좀',
  '보다',
  '날좀',
  '보다'],
 ['박근혜',
  '깜짝',
  '시구',
  '글러브',
  '태극마크',
  '운동화',
  '일제',
  '아식스',
  '이투데이',
  '내',
  '이렇다',
  '줄',
  '알다',
  '팬티',
  '욱일승천기',
  '들다',
  '껄'],
 ['이',
  '무개념',
  '여인',
  '국산',
  '운동화',
  '신기',
  '바',
  '것',
  '정말',
  '지나',
  '친',
  '기대',
  '잖다',
  '박근혜',
  '깜짝',
  '시구',
  '글러브',
  '태극마크',
  '운동화',
  '일제',
  '아식스',
  '이투데이',
  'via'],
 ['선동',
  '가지가지',
  '다',
  '김대중',
  '아식스',
  '괜찮다

In [44]:
texcrapy.tokenize_doc({'text':"today #kicks #asics #gellyte3 #gelyte3 갈라진 혀를 보완하려면 예쁜 양말이 필요하다.. http://instagram.com/p/dD5kkNP4ZL/ "}, with_pos=True, tagger=twitter)

[('today', 'Alpha'),
 ('kicks', 'Alpha'),
 ('asics', 'Noun'),
 ('gellyte', 'Alpha'),
 ('3', 'Number'),
 ('gelyte', 'Alpha'),
 ('3', 'Number'),
 ('갈라', 'Noun'),
 ('진', 'Noun'),
 ('혀', 'Noun'),
 ('보완', 'Noun'),
 ('려면', 'Noun'),
 ('예쁘다', 'Adjective'),
 ('양말', 'Noun'),
 ('필', 'Noun'),
 ('요하', 'Noun')]